<a href="https://colab.research.google.com/github/Nillooffarr/UNet_project/blob/Version1/FinaalUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
from tensorflow.keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from skimage.io import imsave
from skimage import io
#we will undersample our training 2D images later (for memory and speed)
image_rows = int(512)
image_cols = int(512) 
all_files = os.listdir('drive/My Drive/MAT/images')
split = int(0.95 * len(all_files))
train_files = all_files[0:split]
test_files  = all_files[split:]

def create_train_data():
    print('-'*30)
    print('Creating training data...')
    print('-'*30)
    imgs_train = [] 
    masks_train = []  
    for f in train_files:
        mask = Image.open(f'drive/My Drive/MAT/masks/{f}')
        raw = Image.open(f'drive/My Drive/MAT/images/{f}')
        masks_train.append(mask)
        imgs_train.append(raw)
    
    imgs = np.ndarray(
            (len(imgs_train), image_rows, image_cols), dtype=np.uint8
            )
    imgs_mask = np.ndarray(
           (len(masks_train), image_rows, image_cols), dtype=np.uint8
            )
    
    for index, img in enumerate(imgs_train):
        imgs[index, :, :] = img
        
    for index, img in enumerate(masks_train):
        imgs_mask[index, :, :] = img

    np.save('imgs_train.npy', imgs)
    np.save('masks_train.npy', imgs_mask)
    print('Saving to .npy files done.')


def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    masks_train = np.load('masks_train.npy')
    return imgs_train, masks_train


def create_test_data():
    imgs_test = []
    masks_test = []
    print('-'*30)
    print('Creating test data...')
    print('-'*30)
    Image_new = Image.open('drive/My Drive/MAT/images/Y100.png') #to test on an image outside dataset
    for f in test_files:
        mask = Image.open(f'drive/My Drive/MAT/masks/{f[:-4]}.png')
        raw = Image.open(f'drive/My Drive/MAT/images/{f}')
        masks_test.append(mask)
        imgs_test.append(raw)               
    imgst = np.ndarray(
            (len(imgs_test), image_rows, image_cols), dtype=np.uint8
            )
    imgs_maskt = np.ndarray(
            (len(masks_test), image_rows, image_cols), dtype=np.uint8
            )
    imstnn = np.ndarray(( image_rows, image_cols,3), dtype=np.uint8)
    imstnew = np.ndarray(( 1,image_rows, image_cols), dtype=np.uint8)
    imstnn [:,:,:] = Image_new
    imstnew[0,:,:] = imstnn[:,:,0]       
    for index, img in enumerate(imgs_test):
        imgst[index, :, :] = img
        
    for index, img in enumerate(masks_test):
        imgs_maskt[index, :, :] = img
    np.save('imgs_test.npy', imgst)
    np.save('masks_test.npy', imgs_maskt)
    np.save('new_test.npy',imstnew)
    print('Saving to .npy files done.')
    

def load_test_data():
    imgs_test = np.load('imgs_test.npy')
    masks_test = np.load('masks_test.npy')
    new_test = np.load('new_test.npy')
    return imgs_test, masks_test, new_test


if __name__ == '__main__':
    create_train_data()
    create_test_data()
    
    


In [ ]:
    
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from skimage.segmentation import mark_boundaries
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from skimage.exposure import rescale_intensity
from keras.callbacks import History
from skimage import io
import matplotlib.pyplot as plt
K.set_image_data_format('channels_last')  # TF dimension ordering in this code

img_rows = int(512/2)
img_cols = int(512/2)
smooth = 1.
#We divide here the number of rows and columns by two because we undersample our data (We take one pixel over two)


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
#The functions return our metric and loss


def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation = 'relu',padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation = 'relu',padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    

    conv2 = Conv2D(64, (3, 3), activation = 'relu',padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation = 'relu',padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3),activation = 'relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation = 'relu',padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3),activation = 'relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3),  padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation = 'relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation = 'relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2),  padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3),activation = 'relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation = 'relu',padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation = 'relu',padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2),  padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation = 'relu',padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation = 'relu',padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2),  padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation = 'relu',padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation = 'relu',padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    #conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv8) to check what effect the first and layer have
    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model
#The different layers in our neural network model (including convolutions, maxpooling and upsampling)


def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p
#We adapt here our dataset samples dimension so that we can feed it to our network

def train_and_predict():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train = load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std
    #Normalization of the train set

    imgs_mask_train = imgs_mask_train.astype('float32')

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
    #Saving the weights and the loss of the best predictions we obtained

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    history=model.fit(imgs_train, imgs_mask_train, batch_size=6, epochs=100, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_maskt, new_test = load_test_data()
    imgs_test = preprocess(imgs_test)
    imgs_test_new = preprocess(new_test)

    imgs_test = imgs_test.astype('float32')
    mean = np.mean(imgs_test)
    std = np.std(imgs_test)
    imgs_test -= mean
    imgs_test /= std
    #Normalization of the test set
    imgs_test_new = imgs_test_new.astype('float32')
    mean = np.mean(imgs_test_new)
    std = np.std(imgs_test_new)
    imgs_test_new -= mean
    imgs_test_new /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    imgs_mask_new = model.predict(imgs_test_new, verbose=1)
    np.save('imgs_mask_test.npy', imgs_mask_test)
    np.save('imgs_mask_new.npy', imgs_mask_new)
    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    plt.title('Model dice coeff')
    plt.ylabel('Dice coeff')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    #plotting our dice coeff results in function of the number of epochs

    pred_dir = 'preds'
    seg_dir = 'seg'
    if not os.path.exists(seg_dir):
        os.mkdir(seg_dir)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)

    mskn = imgs_mask_new[0][:,:,0]
    a=rescale_intensity(imgs_test_new[0][:,:,0],out_range=(-1,1))
    b=(mskn).astype('uint8')
    mskn[mskn >= 0.5] = 1 
    mskn[mskn < 0.5] = 0 
    io.imsave(os.path.join(pred_dir, 'new_pred.png'),mark_boundaries(a,b))
    combined = np.concatenate([a, mskn, a*mskn], axis = 1)
    io.imsave(os.path.join(seg_dir,  'new_seg.png'),combined)
    for k in range(len(imgs_mask_test)):
        msk = imgs_mask_test[k][:,:,0]
        a=rescale_intensity(imgs_test[k][:,:,0],out_range=(-1,1))
        b=(msk).astype('uint8')
        msk[msk >= 0.5] = 1 
        msk[msk < 0.5] = 0 

        combined = np.concatenate([a, msk, a*msk], axis = 1)
        io.imsave(os.path.join(seg_dir, str(k) + '_seg.png'),combined)
        io.imsave(os.path.join(pred_dir, str(k) + '_pred.png'),mark_boundaries(a,b))


    #Saving our predictions in the directory 'preds' and 'Seg'
